In [ ]:
! pip install pyspark boto3
! pip install -q awscli
! aws configure

In [ ]:
import pandas as pd
import boto3
import json
from boto3.dynamodb.conditions import Key
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

spark = SparkSession.builder.appName("PySpark2").getOrCreate()

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('MARKET_LIST')

response = table.query(
    KeyConditionExpression=Key('PK').eq('USER#b4c894d8-3091-70b2-79f6-78ebfd1b527f')
)
data = response['Items']

pdf = pd.DataFrame(data)
schema = StructType([StructField(col, StringType(), True) for col in pdf.columns])
df = spark.createDataFrame(pdf, schema)

# filtro últimos 6 meses
hoje = datetime.now()
limite = hoje - timedelta(days=6*30)
df = df[df['date'] >= limite]
df.show(truncate=False)

In [ ]:
#filtra e lista as tarefas
df = df.filter(
    ((df.type_task == "Tarefa a Ser Feita") & (df.status == "todo") & (df.date < date_sub(current_date(), 15))) |
    ((df.type_task == "Item de Compra") & (df.status == "todo") & (df.date < date_sub(current_date(), 30)))
)

df = df.withColumn("month", date_format("date", "yyyy-MM"))
df.select("month", "date", "type_task", "name", "status", "PK", "SK").orderBy("month", "date").show(truncate=False)

In [ ]:
# formatar meses
months = []
for i in range(5, -1, -1):
    months.append((hoje - relativedelta(months=i)).strftime("%Y-%m"))
df_months = spark.createDataFrame([(m,) for m in months], ["month"])

df = df_months.join(df, on="month", how="left")

# formatar planilha
df = (
    df.groupBy("month")
      .pivot("type_task", ["Tarefa a Ser Feita", "Item de Compra"])
      .agg(count("*"))
      .na.fill(0)
      .orderBy("month")
)
df.show(truncate=False)

In [ ]:
# gerar .csv
df = df.toPandas()
df.to_csv("abandoned_tasks.csv", index=False)